# Scrape Data using NYT API

### Wharton S&P Data

In [150]:
import yfinance as yf
import pandas as pd
import numpy as np

In [154]:
spy = yf.download("SPY")
spy.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-01-29,43.96875,43.96875,43.75000,43.93750,26.079659,1003200
1993-02-01,43.96875,44.25000,43.96875,44.25000,26.265144,480500
1993-02-02,44.21875,44.37500,44.12500,44.34375,26.320782,201300
1993-02-03,44.40625,44.84375,44.37500,44.81250,26.599014,529400
1993-02-04,44.96875,45.09375,44.46875,45.00000,26.710312,531500


### NYT Data 
src: https://towardsdatascience.com/collecting-data-from-the-new-york-times-over-any-period-of-time-3e365504004

In [3]:
import os
import json
import time
import requests
import datetime
import dateutil
import pandas as pd
import glob
from dateutil.relativedelta import relativedelta

In [5]:
months_in_range = []
for i in range(1996, 2021, 4):
    end = datetime.date(i,11,30)
    start = end - relativedelta(months=2)
    months_in_year = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y %-m").tolist()]
    months_in_range.extend(months_in_year)
months_in_range

[['1996-10-01', '00:00:00'],
 ['1996-11-01', '00:00:00'],
 ['2000-10-01', '00:00:00'],
 ['2000-11-01', '00:00:00'],
 ['2004-10-01', '00:00:00'],
 ['2004-11-01', '00:00:00'],
 ['2008-10-01', '00:00:00'],
 ['2008-11-01', '00:00:00'],
 ['2012-10-01', '00:00:00'],
 ['2012-11-01', '00:00:00'],
 ['2016-10-01', '00:00:00'],
 ['2016-11-01', '00:00:00'],
 ['2020-10-01', '00:00:00'],
 ['2020-11-01', '00:00:00']]

In [9]:
os.getcwd() # Make sure this is the root directory for your repo

'C:\\Users\\smuke\\OneDrive\\Desktop\\NYTvMarket\\NYTvMarket'

In [10]:
def send_request(date):
    '''Sends a request to the NYT Archive API for given date.'''
    base_url = 'https://api.nytimes.com/svc/archive/v1/'
    month = str(int(date[0][5:7]))
    url = base_url + '/' + date[0][0:4] + '/' + month + '.json?api-key=' + "Jha8a5QFDpLMrDm53n5aXK4vAlt5d1Sf"
    print(url)
    response = requests.get(url).json()
    time.sleep(6)
    return response


def is_valid(article, date):
    '''An article is only worth checking if it is in range, and has a headline.'''
    is_in_range = date > start and date < end
    has_headline = type(article['headline']) == dict and 'main' in article['headline'].keys()
    return is_in_range and has_headline


def parse_response(response):
    '''Parses and returns response as pandas data frame.'''
    data = {'headline': [],  
        'date': [], 
        'doc_type': [],
        'material_type': [],
        'section': [],
        'keywords': []}
    
    articles = response['response']['docs'] 
    for article in articles: # For each article, make sure it falls within our date range
        date = dateutil.parser.parse(article['pub_date']).date()
        if is_valid(article, date):
            data['date'].append(date)
            data['headline'].append(article['headline']['main']) 
            if 'section' in article:
                data['section'].append(article['section_name'])
            else:
                data['section'].append(None)
            data['doc_type'].append(article['document_type'])
            if 'type_of_material' in article: 
                data['material_type'].append(article['type_of_material'])
            else:
                data['material_type'].append(None)
            keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
            data['keywords'].append(keywords)
    return pd.DataFrame(data) 


def get_data(dates):
    '''Sends and parses request/response to/from NYT Archive API for given dates.'''
    total = 0
    print('Date range: ' + str(dates[0]) + ' to ' + str(dates[-1]))
    if not os.path.exists('headlines'):
        os.mkdir('headlines')
    for date in dates:
        print(date[5:7])
        response = send_request(date)
        df = parse_response(response)
        total += len(df)
        df.to_csv('headlines/' + date[0][0:4] + '-' + date[0][5:7] + '.csv', index=False)
        print('Saving headlines/' + date[0][0:4] + '-' + date[1][5:7] + '.csv...')
    print('Number of articles collected: ' + str(total))

In [33]:
get_data(months_in_range)
os.chdir('./headlines/')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
combined_csv.to_csv( "NYTData.csv", index=False, encoding='utf-8-sig')
try:
    all_filenames.remove("NYTData.csv")
except ValueError:
    print("Success")
for file in all_filenames:
    os.remove(file)

Date range: ['2020-10-01', '00:00:00'] to ['2020-11-01', '00:00:00']
[]
https://api.nytimes.com/svc/archive/v1//2020/10.json?api-key=Jha8a5QFDpLMrDm53n5aXK4vAlt5d1Sf
Saving headlines/2020-:0.csv...
[]
https://api.nytimes.com/svc/archive/v1//2020/11.json?api-key=Jha8a5QFDpLMrDm53n5aXK4vAlt5d1Sf
Saving headlines/2020-:0.csv...
Number of articles collected: 12764


In [34]:
os.chdir('..')
os.getcwd()

'C:\\Users\\smuke\\OneDrive\\Desktop\\NYTvMarket\\NYTvMarket'